In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,9,10,11,12,13,...,48,49,54,55,56,57,59,60,62,63
2,1109.72,-0.01,-0.02,0.04,-0.03,0.01,-0.01,0.00,0.01,-0.01,...,-0.00,0.00,-0.00,0.00,-0.01,-0.00,-0.00,-0.00,0.00,0.00
3,-0.01,254.35,0.01,0.01,0.01,-0.00,0.00,-0.01,0.00,-0.01,...,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00
4,-0.02,0.01,1215.44,-0.04,0.00,0.01,0.01,-0.04,-0.01,0.00,...,0.00,0.01,-0.00,0.01,0.00,-0.00,0.00,-0.00,0.00,-0.00
5,0.04,0.01,-0.04,852.88,-0.01,0.01,0.02,-0.00,-0.00,0.02,...,0.00,-0.01,-0.00,-0.00,-0.01,0.00,-0.00,0.00,0.00,0.00
6,-0.03,0.01,0.00,-0.01,448.03,-0.01,-0.01,0.01,-0.02,-0.01,...,-0.01,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00
9,0.01,-0.00,0.01,0.01,-0.01,121.56,0.01,-0.01,-0.01,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00
10,-0.01,0.00,0.01,0.02,-0.01,0.01,270.31,0.00,0.00,-0.00,...,-0.00,0.01,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
11,0.00,-0.01,-0.04,-0.00,0.01,-0.01,0.00,374.66,-0.01,0.02,...,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00
12,0.01,0.00,-0.01,-0.00,-0.02,-0.01,0.00,-0.01,249.79,0.01,...,-0.00,-0.00,0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00
13,-0.01,-0.01,0.00,0.02,-0.01,0.00,-0.00,0.02,0.01,288.47,...,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,9,10,11,12,13,...,48,49,54,55,56,57,59,60,62,63
2,1.00000,-0.00001,-0.00002,0.00004,-0.00004,0.00002,-0.00002,0.00000,0.00002,-0.00002,...,-0.00002,0.00001,-0.00002,0.00001,-0.00002,-0.00000,-0.00000,-0.00001,0.00001,0.00000
3,-0.00001,1.00000,0.00002,0.00002,0.00002,-0.00002,0.00000,-0.00002,0.00001,-0.00003,...,-0.00001,-0.00000,0.00001,-0.00000,-0.00002,0.00001,0.00001,0.00001,0.00000,-0.00001
4,-0.00002,0.00002,1.00000,-0.00004,0.00000,0.00002,0.00002,-0.00006,-0.00002,0.00000,...,0.00001,0.00001,-0.00001,0.00002,0.00000,-0.00001,0.00001,-0.00001,0.00000,-0.00000
5,0.00004,0.00002,-0.00004,1.00000,-0.00002,0.00003,0.00003,-0.00000,-0.00001,0.00005,...,0.00001,-0.00001,-0.00001,-0.00001,-0.00002,0.00002,-0.00001,0.00000,0.00000,0.00001
6,-0.00004,0.00002,0.00000,-0.00002,1.00000,-0.00003,-0.00002,0.00003,-0.00005,-0.00002,...,-0.00005,-0.00001,0.00001,-0.00000,-0.00001,-0.00001,-0.00000,0.00001,-0.00000,0.00001
9,0.00002,-0.00002,0.00002,0.00003,-0.00003,1.00000,0.00003,-0.00003,-0.00004,0.00001,...,0.00001,0.00002,0.00004,0.00001,0.00003,0.00002,-0.00000,-0.00001,0.00000,-0.00000
10,-0.00002,0.00000,0.00002,0.00003,-0.00002,0.00003,1.00000,0.00001,0.00001,-0.00001,...,-0.00001,0.00003,0.00003,-0.00001,-0.00001,-0.00000,0.00002,0.00001,-0.00000,0.00000
11,0.00000,-0.00002,-0.00006,-0.00000,0.00003,-0.00003,0.00001,1.00000,-0.00003,0.00005,...,0.00001,-0.00000,-0.00000,-0.00002,0.00001,0.00000,0.00000,0.00001,-0.00001,0.00000
12,0.00002,0.00001,-0.00002,-0.00001,-0.00005,-0.00004,0.00001,-0.00003,1.00000,0.00002,...,-0.00000,-0.00000,0.00002,-0.00002,-0.00004,-0.00000,-0.00001,-0.00001,-0.00000,-0.00000
13,-0.00002,-0.00003,0.00000,0.00005,-0.00002,0.00001,-0.00001,0.00005,0.00002,1.00000,...,0.00000,0.00001,-0.00002,-0.00001,-0.00001,0.00002,0.00001,-0.00001,0.00001,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.24971811021564214

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.24562763e+04 3.51463479e+04 2.08295191e+04 1.63972907e+04
 7.79654166e+03 5.75150994e+03 4.19684197e+03 4.05494250e+03
 3.73399560e+03 2.39608566e+03 2.10624901e+03 1.90754601e+03
 1.86373562e+03 1.70975005e+03 1.45245993e+03 1.27705761e+03
 1.06874182e+03 9.48268465e+02 8.63452304e+02 7.78992055e+02
 6.45557565e+02 5.90193958e+02 5.02027636e+02 4.14070277e+02
 3.66181624e+02 2.93278052e+02 2.13571141e+02 1.78118449e+02
 1.33502087e+02 8.91417613e+01 5.97176697e+01 3.30864224e+01
 1.11300721e+01 4.58156373e-01 7.57220630e-27]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,-0.144067,-0.005055,0.988375,-0.032843,-0.009893,-0.002333,-0.005406,-0.007946,-0.004984,-0.005824,...,-0.001129,-0.004187,-0.000519,-0.001444,-0.003322,-0.000954,-0.002088,-0.000064,-0.000767,-0.000085
1,0.986842,-0.007306,0.139872,-0.059028,-0.014665,-0.003333,-0.007829,-0.011533,-0.007145,-0.008437,...,-0.001621,-0.006015,-0.000742,-0.002062,-0.004772,-0.001359,-0.002989,-0.000092,-0.001093,-0.000121
2,0.047028,-0.008775,0.035274,0.988818,-0.019683,-0.003886,-0.009437,-0.014691,-0.008613,-0.010226,...,-0.001871,-0.007172,-0.000858,-0.002402,-0.005623,-0.001569,-0.003486,-0.000106,-0.001268,-0.000140
3,0.039565,-0.012440,0.031675,0.120465,-0.030146,-0.005366,-0.013415,-0.021562,-0.012114,-0.014654,...,-0.002577,-0.009999,-0.001172,-0.003304,-0.007778,-0.002163,-0.004794,-0.000146,-0.001741,-0.000193
4,0.015223,-0.016871,0.013323,0.024563,-0.092692,-0.006456,-0.018706,-0.039974,-0.016410,-0.021058,...,-0.002999,-0.012905,-0.001367,-0.003884,-0.009620,-0.002521,-0.005718,-0.000170,-0.002037,-0.000224
5,0.012240,-0.022092,0.010790,0.018295,0.985094,-0.007675,-0.025148,-0.077162,-0.021446,-0.029280,...,-0.003537,-0.016194,-0.001587,-0.004544,-0.011721,-0.002945,-0.006759,-0.000195,-0.002367,-0.000259
6,0.003025,-0.009703,0.002677,0.004292,0.023028,-0.002813,-0.011578,-0.479054,-0.009159,-0.014537,...,-0.001281,-0.006479,-0.000565,-0.001638,-0.004476,-0.001069,-0.002473,-0.000069,-0.000846,-0.000094
7,0.004941,-0.016968,0.004463,0.007012,0.034595,-0.004830,-0.020255,0.729895,-0.016220,-0.025585,...,-0.002150,-0.011272,-0.000960,-0.002795,-0.007667,-0.001802,-0.004219,-0.000118,-0.001440,-0.000158
8,0.016739,-0.068146,0.014997,0.023699,0.101131,-0.018070,-0.083693,0.464764,-0.064379,-0.109821,...,-0.008019,-0.043558,-0.003598,-0.010424,-0.029166,-0.006687,-0.015771,-0.000441,-0.005362,-0.000586
9,0.005653,-0.088869,0.005086,0.007678,0.021964,-0.010642,-0.188247,0.036968,-0.076373,0.965123,...,-0.004448,-0.034589,-0.001978,-0.005850,-0.019161,-0.003684,-0.009137,-0.000243,-0.002941,-0.000320


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([1.14406711, 1.00730565, 0.96472636, 0.8795349 , 1.09269194,
       1.0076755 , 1.01157834, 0.27010466, 1.06437907, 0.03487715,
       1.02404197, 0.99892332, 0.98603787, 0.98405596, 1.03666232,
       1.27604089, 1.01474608, 1.01835429, 1.00725311, 1.07989376,
       0.99518627, 1.58193396, 0.9758625 , 0.97715717, 1.02105966,
       1.09721366, 0.97154724, 1.01445564, 0.42419205, 0.96009417,
       0.32071765, 0.91685752, 1.09625725, 0.98076121, 0.40577162])